In [ ]:
# import sphinxai is not available in this environment
import os
import sys
import json
import asyncio

# Read the target text from the environment (passed by FastAPI)
target_text = os.environ.get('SPHINX_TARGET_TEXT', '')
output_file = os.environ.get('SPHINX_OUTPUT_FILE', 'sphinx_output.json')

async def analyze_text(text):
    if not text:
        return {"risk_level": "unknown", "trust_score": 0, "reasoning_summary": "No text provided."}
        
    # Standard Prompt format for Trust Signals
    prompt = f"""You are a Trust and Safety AI. Analyze the following text and determine if it looks like a fake account, spam, scam, or harmful content.
    
Text to analyze:
\"{text}\"

Return the response in JSON format with exact keys:
- risk_level: ('low', 'medium', or 'high')
- trust_score: (0-100 integer, 100 being completely safe)
- confidence score: (0-100 integer, 100 being very confident in the risk level and reasoning)
- reasoning_summary: (1 to 2 sentences explaining your decision)
"""
    
    try:
        # The magic sphinxai library is auto-injected when running via sphinx-cli
        response = await sphinxai.llm(
            prompt=prompt,
            model_size="M",  # Use M for balanced speed and quality
            timeout=30.0
        )
        
        # Clean the block if it returns markdown json fences
        cleaned_response = response.strip()
        if cleaned_response.startswith('```json'):
            cleaned_response = cleaned_response[7:-3]
        elif cleaned_response.startswith('```'):
            cleaned_response = cleaned_response[3:-3]
            
        return json.loads(cleaned_response)
    except Exception as e:
        print(f"Sphinx llm execution error: {str(e)}", file=sys.stderr)
        return {"risk_level": "high", "trust_score": 0, "reasoning_summary": f"Error during evaluation: {str(e)}"}

# Run the analysis
result = await analyze_text(target_text)

# Save the result to the output file for FastAPI to read
with open(output_file, 'w') as f:
    json.dump(result, f)

print(f"Analysis complete. Saved to {output_file}")


ModuleNotFoundError: No module named 'sphinxai'

In [0]:
import